# Setup

In [2]:
from datetime import datetime
from packaging import version

import tensorflow as tf
from tensorflow.keras import(
    layers,
    Sequential,
)

import numpy as np

In [4]:
print(f'Tensorflow version: {tf.__version__}')
assert version.parse(tf.__version__).release[0] >= 2,\
    'This notebook requires TensorFlow 2.0 or above'

Tensorflow version: 2.8.0


# Setup Data for a Simple Regression

In [ ]:
data_size = 1000
train_perc = 0.8
train_size = int(data_size * train_perc)

x = np.linspace(-1, 1, data_size)
np.random.shuffle(x)

y = 0.5 * x + 2 + np.random.normal(0, 0.05, (data_size, ))

x_train, y_train = x[:train_size], y[:train_size]
x_test, 